In [1]:
#!pip install foursquare
#!pip install folium
#!pip install geopy

In [2]:
from pandas.io.json import json_normalize
import folium
from geopy.geocoders import Nominatim 
import requests
import pandas as pd
import folium
from folium import plugins
import math

In [3]:
geolocator = Nominatim(user_agent="foursquare_agent")

ny_location = geolocator.geocode("New York, USA")
ny_latitude = ny_location.latitude
ny_longitude = ny_location.longitude

print("New York latitude: ", ny_latitude, ", New York longitude: ", ny_longitude)

New York latitude:  40.7127281 , New York longitude:  -74.0060152


In [4]:
CLIENT_ID = 'YTSV01M350Z0VJDDS5AE2AT5TYRI1M44PREW5G2IPLZZB0V3' # your Foursquare ID
CLIENT_SECRET = 'TJQPHVQFM4MODEUAPH5ABIS4JPGD3XKZ1T2NPOE3YB3MZIEC' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
radius= 20000

In [5]:
def searchQuery(keywords, latitude, longitude):
    dataframe = pd.DataFrame()
    rs = []
    details = []
    for word in keywords:  
        search_query = word
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
        results = requests.get(url).json()
        rs.append(results)
        venues = results['response']['venues']
        dataframe = pd.concat([dataframe, json_normalize(venues)],sort=True)
        for venue in venues:
            venue_id = venue['id']
            url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
            detail = requests.get(url).json()
            details.append(detail)
    return dataframe,rs,details

dfny,rsny,details = searchQuery(['coffee'], ny_latitude, ny_longitude) 

<ipython-input-5-7d44987bb24a>:11: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = pd.concat([dataframe, json_normalize(venues)],sort=True)


In [6]:
#print(details[0]['response']['venue'])
#for i,d in enumerate(details):
#    details[i] = d['response']['venue']

In [7]:
pd.set_option('display.max_columns', 500)

In [8]:
#dfny_details = pd.io.json.json_normalize(dfny['details'])
#dfny = pd.concat([dfny, dfny_details], axis=1)
#dfny

In [9]:
dfny.to_csv('newyorkcoffeewithdetails.csv')

In [10]:
link='C:\\Users\\user\\Desktop\\EE393\\Project\\newyorkcoffeewithdetails.csv'
df=pd.read_csv(link,error_bad_lines=False)
df.head()

,Unnamed: 0,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,0,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",2368674.0,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/for-five-co...,False,5dba113267232c000803cb37,NaN,US,New York,United States,NaN,752,"['New York, NY 10280', 'United States']","[{'label': 'display', 'lat': 40.71224, 'lng': ...",40.712240,-74.014915,NaN,10280.0,NY,For Five Coffee Roasters,v-1609515053,NaN
1,1,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,49c79540f964a520af571fe3,76 Chambers St,US,New York,United States,at Broadway,202,"['76 Chambers St (at Broadway)', 'New York, NY...","[{'label': 'display', 'lat': 40.71442758460976...",40.714428,-74.006859,NaN,10007.0,NY,Blue Spoon Coffee Co.,v-1609515053,NaN
2,2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",2368673.0,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/for-five-coffee-...,False,5cf965947564f7002c79aa21,1 Liberty Plz,US,New York,United States,Broadway,522,"['1 Liberty Plz (Broadway)', 'New York, NY 100...","[{'label': 'display', 'lat': 40.709554, 'lng':...",40.709554,-74.010576,NaN,10006.0,NY,For Five Coffee Roasters,v-1609515053,NaN
3,3,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",2461479.0,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/new-kam-hing-118...,False,4a118d01f964a52023771fe3,118 Baxter St,US,New York,United States,btwn Hester and Canal St,822,"['118 Baxter St (btwn Hester and Canal St)', '...","[{'label': 'display', 'lat': 40.71801668499095...",40.718017,-73.999210,NaN,10013.0,NY,Kam Hing Coffee Shop 金興,v-1609515053,53113426.0
4,4,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,583af456ce593d41644bd7a6,120 Lafayette St Frnt C,US,New York,United States,NaN,811,"['120 Lafayette St Frnt C', 'New York, NY 1001...","[{'label': 'display', 'lat': 40.71863256946489...",40.718633,-74.000367,NaN,10013.0,NY,Kaigo Coffee Room,v-1609515053,NaN


In [13]:
locationnew= pd.DataFrame(columns=['LATITUDE', 'LONGITUDE','NAME'])
lat = float(input("Enter the latitude of your place: "))
lon = float(input("Enter the longitude of your place: "))
for i in range(len(df)):
    R = 6373.0

    lat1 = math.radians(lat)
    lon1 = math.radians(lon)
    lat2 = math.radians(df["location.lat"].iloc[i])
    lon2 = math.radians(df["location.lng"].iloc[i])

    dlon = lon2 - lon1

    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance =R*c
    if (distance<1.8):
        locationnew=locationnew.append({'LATITUDE' : df["location.lat"].iloc[i],'LONGITUDE' : df["location.lng"].iloc[i],"NAME": df["name"].iloc[i]} , ignore_index=True)

print(locationnew.head())



Enter the latitude of your place:  40.709554
Enter the longitude of your place:  -74.010576


    LATITUDE  LONGITUDE                      NAME
0  40.712240 -74.014915  For Five Coffee Roasters
1  40.714428 -74.006859     Blue Spoon Coffee Co.
2  40.709554 -74.010576  For Five Coffee Roasters
3  40.718017 -73.999210   Kam Hing Coffee Shop 金興
4  40.718633 -74.000367         Kaigo Coffee Room


In [ ]:
#latitude_thr_max=40.75
#latitude_thr_min=40.7
#longitude_thr_max=-73.9
#longitude_thr_min=-74
#for i in range(len(locaiton)):
    #if (longitude_thr_min<df["location.lng"].iloc[i]<longitude_thr_max) & (latitude_thr_min<df["location.lat"].iloc[i]<latitude_thr_max):
    #    locationnew=locationnew.append({'LATITUDE' : df["location.lat"].iloc[i],'LONGITUDE' : df["location.lng"].iloc[i],"NAME": df["name"].iloc[i]} , ignore_index=True)


In [14]:
new_york_location=[40.730610,-73.935242]
m=folium.Map(new_york_location,zoom_start=11,width="%100",height="%100")
for i in range(len(locationnew)):
    folium.Marker(locationnew.loc[i, ['LATITUDE','LONGITUDE']],
                  tooltip=locationnew["NAME"].iloc[i],
                 ).add_to(m)
m